In [1]:
import pandas as pd
import numpy as np
from math import isnan
from scipy.stats import lognorm

In [2]:
def MeanOfLn(low_bnd, up_bnd):
    """
    The algorithm for mean of the log normal distribution
    * Input
        low_bnd - The lower bounds of the distribution
        up_bnd - The upper bounds of the distribution
    * Output
        Returns the log normal mean
    """
    return (np.log(up_bnd) + np.log(low_bnd)) / 2
def StdDevOfLn(low_bnd, up_bnd):
    """
    The algorithm for standard deviation of the log normal distribution
    * Input
        low_bnd - The lower bounds of the distribution
        up_bnd - The upper bounds of the distribution
    * Output
        Returns the log normal standard deviation
    """
    return (np.log(up_bnd) - np.log(low_bnd)) / 3.29
def NormInv(low_bnd, up_bnd):
    """
    The algorithm to generate a log normal distribution
    * Input
        low_bnd - The lower bounds of the distribution
        up_bnd - The upper bounds of the distribution
    * Output
        Returns the a random normalized value within the specified range
    """
    p = np.random.rand()
    mean = MeanOfLn(up_bnd, low_bnd)
    std_dev = StdDevOfLn(up_bnd, low_bnd)
    return lognorm.ppf(p, std_dev, loc=0, scale=np.exp(mean))
def DidOccur(event_prob, low_bnd, up_bnd):
    """
    This function simulates if an event occured

    * Input
        event_prob - The probability the event will occur
    * Output
        Returns the cost of impact in dollars
    """
    if np.random.rand() < event_prob:
        result = NormInv(low_bnd, up_bnd)
        if not isnan(result):
            return int(result)
    return 0
def SimulatedEvent(num_reps):
    """
    This is an example of how to generate random events and impacts. The example is based on a lognormal
    distriution rang of $0 to $5 million. The range of the events are between .1 and .01.
    * Input
        num_reps - The number of outcomes to simulate
    * Output
        Returns the total cost for all simulated outcomes
    """
    low_bnd_min, low_bnd_max = 50000, 1500000
    up_bnd_min, up_bnd_max = 500000, 5000000
    # The probabilities
    events = np.random.uniform(.1, 0, num_reps).round(2)
    lower = np.random.randint(low_bnd_min, low_bnd_max, num_reps)
    upper = np.random.randint(up_bnd_min, up_bnd_max, num_reps)
    df = pd.DataFrame(index=range(num_reps), data={'Annual_Event_Probability': events,
                                                    'Lower_Bound': lower,
                                                    'Upper_Bound': upper})
    df['Random_Result'] = df.apply(lambda x: DidOccur(x.Annual_Event_Probability, x.Lower_Bound, x.Upper_Bound), axis=1)
    return df['Random_Result'].sum()

In [3]:
scenarios = []
for i in range(0,10000):
    scenarios.append(SimulatedEvent(40))
df = pd.DataFrame.from_dict(scenarios)

In [4]:
avg = (df.sum() / 10000).round(2)
avg

0    155790.87
dtype: float64